In [ ]:
import ipywidgets as widgets
import pandas as pd
import requests
import json
from IPython.display import clear_output
from sklearn.externals import joblib

counter = 0
scores = []
# modelTest =joblib.load('movielens_sar_model.pkl')
mostPopular =modelTest.get_popularity_based_topk(top_k=30,sort_top_k=True).join(data[['MovieId', 'Title']].drop_duplicates().set_index('MovieId'), 
                                on='MovieId', 
                                how='inner')[['MovieId','Title']].sample(5)
style = {'description_width': 'initial'}
# title= widgets.Label("Let's rate a few popular movies",value = r'\(\color{red} {highlighted}\)')
title= widgets.Label("Let's rate a few popular movies")
movie =widgets.RadioButtons(
    options=[1.0,2.0,3.0,4.0,5.0],
    value=3.0,
    description= str(counter+1) + ". " +str(mostPopular.iloc[0,1])+ ':',
    disabled=False,
    style=style
)
button = widgets.Button(description='Next', style = style)
def on_button_clicked(b):
    global counter
    global scores 
    global movie
    global button
    global title

    scores.append(movie.value)
    if counter <4:
        movie =widgets.RadioButtons(
        options=[1.0,2.0,3.0,4.0,5.0],
        value=3.0,
        description= str(counter+2) + ". " + str(mostPopular.iloc[counter+1,1])+ ':',
        disabled=False,
        style=style
        )
        vbox.children = [title,movie,button]

    if counter ==3:
        button.description = "Submit"
        button.style.button_color = 'lightgreen'
    if counter ==4:
        clear_output()
        title = widgets.Label("Here are the recommended movies based on your ratings.", style=style)
        display(title)
        d = {'MovieId': mostPopular['MovieId'].tolist(), 'Rating': scores}
        df = pd.DataFrame(data=d).to_json()
        test_sample_encoded = bytes(df,encoding = 'utf8')
        scoringURI = "http://52.147.184.63:80/score"
        headers = {'content-type': 'application/json'}
        # sending post request and saving response as response object 
        similar = requests.post(url = scoringURI, data=test_sample_encoded, headers=headers).json()
        # similar =service.run(input_data = test_sample_encoded)
        temp = pd.read_json(similar).join(data[['MovieId', 'Title']].drop_duplicates().set_index('MovieId'), 
                                on='MovieId', 
                                how='inner').sort_values(by=['prediction'], ascending=False)
        display(temp)
    counter +=1
        
button.on_click(on_button_clicked)
vbox = widgets.VBox([title,movie,button])
display(vbox)